## Example: Developing a Bernoulli Binary Bandit Ticker Picker Agent
In this example, we'll develop an agent to select which stocks we should include in a portfolio $\mathcal{P}$ by solving a Multi-arm Binary Bandit problem using [ϵ-Greedy Thompson Sampling](https://arxiv.org/abs/1707.02038). 

### Problem
* Let's have `N` agents independently analyze sequences of daily Open High Low Close (OHLC) data and rank-order their belief that ticker `XYZ` will return at least the risk-free rate in the next step. 
* We then sample the `world`. If ticker `XYZ` returns greater than or equal to the risk-free rate in the next time period, the agent receives a reward of `+1`. Otherwise, the agent recives a reward of `0`.
* Each agent develops a distribution of beliefs based on this experimentation, which is stored in a $\beta$-distribution
* Each ticker is an action in the set $\mathcal{A}=\left\{a_{1},a_{2},\dots,a_{K}\right\}$

## Setup

In [1]:
include("Include.jl");

    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
   Resolving package versions...
  No Changes to `~/Desktop/julia_work/CHEME-5660-Examples-F23/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Examples-F23/Manifest.toml`
  Activating project at `~/Desktop/julia_work/CHEME-5660-Examples-F23`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Examples-F23/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Examples-F23/Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Examples-F23/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Examples-F23/Manifest.toml`


In [2]:
include(joinpath(_PATH_TO_SRC, "CHEME-5660-L14a-BanditProblems-CodeLibrary.jl"));

## Prerequisites: Load and clean the historical dataset
We gathered a daily open-high-low-close `dataset` for each firm in the [S&P500](https://en.wikipedia.org/wiki/S%26P_500) since `01-03-2018` until `11-17-2023`, along with data for a few exchange traded funds and volatility products during that time. 

In [3]:
original_dataset = load(joinpath(_PATH_TO_DATA, 
        "SP500-Daily-OHLC-1-3-2018-to-11-17-2023.jld2")) |> x-> x["dataset"];

### Clean the data
Not all of the tickers in our dataset have the maximum number of trading days for various reasons, e.g., acquistion or de-listing events. Let's collect only those tickers with the maximum number of trading days.

* First, let's compute the number of records for a company that we know has a maximim value, e.g., `AAPL` and save that value in the `maximum_number_trading_days` variable:

In [4]:
maximum_number_trading_days = original_dataset["AAPL"] |> nrow;

Now, lets iterate through our data and collect only those tickers that have `maximum_number_trading_days` records. Save that data in the `dataset::Dict{String,DataFrame}` variable:

In [5]:
dataset = Dict{String,DataFrame}();
for (ticker,data) ∈ original_dataset
    if (nrow(data) == maximum_number_trading_days)
        dataset[ticker] = data;
    end
end
dataset;

Let's get a list of firms that we have in cleaned up `dataset`, and save it in the `all_tickers` array:

In [6]:
all_tickers = keys(dataset) |> collect |> sort;
K = length(all_tickers)

459

## Initialize the `world` function

In [7]:
function world(action::Int64, start::Int64, data::Dict{String, DataFrame}, tickers::Array{String,1}; 
        Δt::Float64 = (1.0/252.0), risk_free_rate::Float64 = 0.05, buffersize::Int64 = 1)::Int64

    # initialize -
    result_flag = 0;

    # daily risk free rate -
    T = buffersize*Δt;

    # grab the ticker we are looking at?
    ticker_symbol = tickers[action];
    
    # compute the expected return over the horizon -
    price_df = data[ticker_symbol];
    time = range(start+1,(start+buffersize), step=1) |> collect
    buffer = Array{Float64,1}();
    for t ∈ time
        P₁ = price_df[t-1,  :volume_weighted_average_price]
        P₂ = price_df[t, :volume_weighted_average_price]
        R = (1/Δt)*log(P₂/P₁);
        push!(buffer,R);
    end
    μ = mean(buffer);

    # if we invested 1 USD, in each how much would we have at the end of horizon -
    W_risk_free = exp(risk_free_rate*T);
    W_ticker = exp(μ*T);
    
    # are we better or worse relative to the risk-free investment?
    if (W_ticker >= W_risk_free)
        result_flag = 1;
    end

    # default -
    return result_flag;
end;

In [8]:
risk_free_rate = 0.047;

## Setup the `ticker-picker` agent

First, let's specify the tickers that we want to examine in the `tickers` array, and store the number of tickers in the `K` variable:

In [9]:
tickers = ["MRK", "JNJ", "MET", "NFLX", "K", "AAPL", "AMD", "MU", "INTC", "MSFT", "SPY", "SPYD"];
K = length(tickers);

Next, we construct the `EpsilonSamplingModel` instance which holds information about the ϵ-greedy sampling approach. The `EpsilonSamplingModel` type has one additional field, the `ϵ` field which controls the approximate fraction of `exploration` steps the algorithm takes; `exploration` steps are purely random.

In [10]:
model = EpsilonSamplingModel()
model.K = K; # tickers
model.α = ones(K); # initialize to uniform values
model.β = ones(K); # initialize to uniform values
model.ϵ = 0.2;

## A single `ticker-picker` agent learning its preferences

In [11]:
time_sample_results_dict_eps = sample(model, world, dataset, tickers; horizon = (maximum_number_trading_days - 1), risk_free_rate = risk_free_rate)

Dict{Int64, Matrix{Float64}} with 1479 entries:
  1144 => [2.0 7.0; 17.0 23.0; … ; 129.0 91.0; 43.0 37.0]
  1175 => [2.0 7.0; 17.0 23.0; … ; 137.0 96.0; 44.0 37.0]
  719  => [1.0 6.0; 11.0 14.0; … ; 69.0 46.0; 29.0 27.0]
  1028 => [2.0 6.0; 17.0 21.0; … ; 119.0 78.0; 37.0 34.0]
  699  => [1.0 6.0; 11.0 13.0; … ; 67.0 45.0; 28.0 26.0]
  831  => [1.0 6.0; 13.0 14.0; … ; 88.0 58.0; 31.0 30.0]
  1299 => [5.0 8.0; 18.0 23.0; … ; 152.0 109.0; 50.0 42.0]
  1438 => [6.0 10.0; 18.0 23.0; … ; 178.0 122.0; 58.0 47.0]
  1074 => [2.0 6.0; 17.0 21.0; … ; 120.0 85.0; 40.0 35.0]
  319  => [1.0 4.0; 4.0 7.0; … ; 29.0 16.0; 15.0 12.0]
  687  => [1.0 6.0; 11.0 13.0; … ; 66.0 45.0; 27.0 26.0]
  1199 => [2.0 7.0; 17.0 23.0; … ; 141.0 98.0; 44.0 37.0]
  185  => [1.0 3.0; 3.0 4.0; … ; 11.0 7.0; 4.0 5.0]
  823  => [1.0 6.0; 13.0 14.0; … ; 87.0 57.0; 31.0 30.0]
  1090 => [2.0 6.0; 17.0 21.0; … ; 121.0 87.0; 41.0 35.0]
  420  => [1.0 4.0; 4.0 8.0; … ; 39.0 24.0; 18.0 15.0]
  1370 => [5.0 9.0; 18.0 23.0; … ; 164

In [12]:
Z = time_sample_results_dict_eps[100]

12×2 Matrix{Float64}:
 1.0  3.0
 1.0  3.0
 2.0  4.0
 2.0  2.0
 1.0  3.0
 9.0  4.0
 9.0  6.0
 7.0  5.0
 3.0  3.0
 8.0  7.0
 7.0  5.0
 4.0  4.0

## Many `ticker-picker` agents learning 

In [13]:
number_of_agents = 100;
trading_day_index = 1469

pref_array = Array{Float64,2}(undef, K, number_of_agents);
agent_specific_data = Array{Beta,2}(undef, K, number_of_agents);

for agent_index ∈ 1:number_of_agents
    
    # sample -
    time_sample_results_dict_eps = sample(model, world, dataset, tickers; horizon = (maximum_number_trading_days - 1), risk_free_rate = risk_free_rate);
    beta_array = build_beta_array(time_sample_results_dict_eps[trading_day_index]);

    # grab data for this agent -
    for k = 1:K
        agent_specific_data[k, agent_index] = beta_array[k]
    end
end

In [14]:
agent_specific_data[1,:] # data for each agent for ticker i 

100-element Vector{Beta}:
 Beta{Float64}(α=26.0, β=33.0)
 Beta{Float64}(α=24.0, β=31.0)
 Beta{Float64}(α=5.0, β=13.0)
 Beta{Float64}(α=40.0, β=41.0)
 Beta{Float64}(α=31.0, β=31.0)
 Beta{Float64}(α=25.0, β=27.0)
 Beta{Float64}(α=50.0, β=56.0)
 Beta{Float64}(α=88.0, β=73.0)
 Beta{Float64}(α=100.0, β=77.0)
 Beta{Float64}(α=102.0, β=86.0)
 Beta{Float64}(α=107.0, β=92.0)
 Beta{Float64}(α=35.0, β=32.0)
 Beta{Float64}(α=65.0, β=57.0)
 ⋮
 Beta{Float64}(α=19.0, β=28.0)
 Beta{Float64}(α=69.0, β=62.0)
 Beta{Float64}(α=14.0, β=18.0)
 Beta{Float64}(α=28.0, β=28.0)
 Beta{Float64}(α=91.0, β=77.0)
 Beta{Float64}(α=4.0, β=10.0)
 Beta{Float64}(α=10.0, β=19.0)
 Beta{Float64}(α=14.0, β=21.0)
 Beta{Float64}(α=11.0, β=19.0)
 Beta{Float64}(α=43.0, β=44.0)
 Beta{Float64}(α=39.0, β=35.0)
 Beta{Float64}(α=44.0, β=43.0)

## The wisdom of the collective
Now that we have prefernces for the `N` agents (encoded as `beta` distributions for each ticker), let's develop a concencous belief in which tickers to include in our portfolio $\mathcal{P}$.

In [26]:
preference_rank_array = Array{Int,2}(undef, number_of_agents, K);
for agent ∈ 1:number_of_agents
        
    # ask agent about thier preference for ticker i -
    experience_distributions = agent_specific_data[:,agent]
    preference_vector = preference(experience_distributions, tickers, N = 5000)
    
    # compute the rank -
    preference_rank = tiedrank(preference_vector, rev=true) .|> x-> trunc(Int64,x) # trunc function is cool!
    for i ∈ 1:K
        preference_rank_array[agent, i] = preference_rank[i];
    end
end

### Build a preference table

In [27]:
preference_rank_array

100×12 Matrix{Int64}:
  8   3   9  10   7   1   4  11   6   2   5  12
 11   7   1   3   2  12  10   9   8   6   5   4
 12   9   6   8  10   5   3   2   7   4   1  11
  8  12  11   3   9  10   4   5   6   1   2   7
  5   2   7   3  12   4   9  10   6  11   1   8
  9   8  11  12   3   1   5   6  10   6   2   4
  8   1   6   9  12   2  10  11   7   5   3   4
  2  10   8   9  12   1   4   6  11   5   3   7
  3   7  11  12   8   2   9   1  10   4   6   5
  2   9  12   1   8   5   4   2  11   7   6  10
  2   8  11   5   1   7  10   3  12   9   4   6
  7   4   9   2  12   3  11   6   5   8   1  10
  2   8   6  11   4  12   3   9   7   1   5  10
  ⋮                   ⋮                   ⋮  
 10   8   2   5   1  12   6   4  11   9   3   7
  4   9   2   5  12  10   1  11   7   3   6   8
  7   9   2   8  12  10  11   5   4   6   1   3
  7   8   4   6  12   1   2  10   5   9   3  11
  2   3   6   8   5  12  10   1   4   7  11   9
 12   9  10   4   6   3   7   8   5   2   1  11
 11   1   7   6   5 

In [44]:
freq = findall(x->x==3,preference_rank_array[:,12]) |> x-> length(x) |> x-> x/number_of_agents

0.09